In [4]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df_sch, meta_sch = pyreadstat.read_sav('data/CY08MSP_STU_QQQ.sav')

KeyboardInterrupt: 

In [ ]:
import os
with open('data/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [145]:
   
cols = utils.meta2docs(meta_sch, excluded=['test', 'CNTRYID'])

cols_vectorstore = Chroma.from_documents(cols, OpenAIEmbeddings(), persist_directory="./chroma_db_cols_vectorstore")

cols_retriever = cols_vectorstore.as_retriever()

In [146]:
len(meta_sch.column_names)

1279

In [147]:
len(cols_vectorstore.get()['ids'])

1278

In [138]:
len(cols)


1278

In [106]:
test_data = [
    {'question':'Which country has the highest average  teacher student ratio? How much is it? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'Which country has the highest average class sizes? How much is it?','expected_columns': ['CLSIZE', 'CNT'], 'expected_answer': 42.65 },    
    {'question':'In how many languages ​​was the questionnaire filled out?', 'expected_columns': ['LANGTEST_QQQ'], 'expected_answer': 54},
    {'question':'What is Hungary average teacher-student ratio?', 'expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 10.83},
    {'question':'What is OECD average teacher-student ratio?', 'expected_columns': ['STRATIO', 'OECD'], 'expected_answer':  12.551608},
    {'question':'Where is average teacher-student ratio in Hungary compared to the OECD average in terms of average teacher-student ratio? Give the difference as number.','expected_columns': ['STRATIO', 'OECD', 'CNT'], 'expected_answer': -1.721608 },    
    #{'question':'how many education systems were involved in the survey? ','expected_columns': ['STRATIO', 'CNT'], 'expected_answer': 28.95},
    {'question':'how many countries were involved in the survey?','expected_columns': ['CNT'], 'expected_answer': 80},
    {'question':'how many countries that are OECD memeber were involved in the survey? ','expected_columns': ['OECD', 'CNT'], 'expected_answer': 37},
    {'question':'how many schools were involved in the survey?','expected_columns': ['CNTSCHID'], 'expected_answer': 21629},
    {'question':'what is the relation between school autonomy  and teacher participation? Please give linear regression coefficient as result','expected_columns': ['SCHAUTO','TCHPART'], 'expected_answer':  0.2216 },
]


In [107]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [108]:
cols_retriever.invoke('1. Student ID')

[Document(page_content='Intl. Student ID', metadata={'original_col_name': 'CNTSTUID', 'year': 2022}),
 Document(page_content='Intl. School ID', metadata={'original_col_name': 'CNTSCHID', 'year': 2022}),
 Document(page_content="Students' age", metadata={'original_col_name': 'AGE', 'year': 2022}),
 Document(page_content="Student's county of birth", metadata={'original_col_name': 'COBN_S', 'year': 2022})]

In [110]:
utils.pipeline(llm, 'How many students were interviewed?', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Interview Date
3. Interview Time
4. Interviewer Name
5. Interview Location
6. Interview Duration
7. Interview Outcome (e.g. pass, fail, pending)
8. Interview Feedback
9. Interview Notes
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['VER_DAT', 'ST330Q02WA', 'ST330Q03WA', 'Option_TQ']
['VER_DAT', 'ST330Q02WA', 'ST330Q03WA', 'Option_TQ']
['IC177Q04JA', 'SISCO', 'ST330Q02WA', 'ST330Q03WA']
['IC177Q04JA', 'SISCO', 'ST330Q02WA', 'ST330Q03WA']
['Option_TQ', 'LANGTEST_QQQ', 'Option_PQ', 'Option_WBQ']
['Option_TQ', 'LANGTEST_QQQ', 'Option_PQ', 'Option_WBQ']
['ST330Q02WA', 'ST330Q03WA', 'Option_TQ', 'SISCO']
['ST330Q02WA', 'ST330Q03WA', 'Option_TQ', 'SISCO']
['ST273Q05JA', 'LANGTEST_QQQ', 'Option_TQ', 'ST327Q06JA']
['ST273Q05JA', 'LANGTEST_QQQ', 'Option_TQ', 'ST327Q06JA']
['PV7MPIN', 'Option_TQ', 'ST327Q07JA', 'PV6MPIN']
['PV7MPIN', 'Option_TQ', 'ST327Q07JA', 'PV6MPIN']
['IC175Q02JA', 'IC175Q01JA', 'Option_TQ', 'LANGTEST_QQQ']
['IC175Q02JA'

{'question': 'How many students were interviewed?',
 'result': 613744.0,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'VER_DAT',
  'ST330Q02WA',
  'ST330Q03WA',
  'Option_TQ',
  'IC177Q04JA',
  'SISCO',
  'ST330Q02WA',
  'ST330Q03WA',
  'Option_TQ',
  'LANGTEST_QQQ',
  'Option_PQ',
  'Option_WBQ',
  'ST330Q02WA',
  'ST330Q03WA',
  'Option_TQ',
  'SISCO',
  'ST273Q05JA',
  'LANGTEST_QQQ',
  'Option_TQ',
  'ST327Q06JA',
  'PV7MPIN',
  'Option_TQ',
  'ST327Q07JA',
  'PV6MPIN',
  'IC175Q02JA',
  'IC175Q01JA',
  'Option_TQ',
  'LANGTEST_QQQ',
  'Option_TQ',
  'ST330Q02WA',
  'LANGTEST_QQQ',
  'PV9READ']}

df_sch['CNTSTUID'].nunique()

In [77]:
float(df_sch['CNTSTUID'].nunique())

613744.0

In [111]:
utils.pipeline(llm, 'How many countries where in the survey?', df_sch, meta_sch, cols_retriever)

1. Country Name
2. Country Code
['CNT', 'COBN_F', 'REGION', 'COBN_M']
['CNT', 'COBN_F', 'REGION', 'COBN_M']
['CNT', 'NatCen', 'SUBNATIO', 'PROGN']
['CNT', 'NatCen', 'SUBNATIO', 'PROGN']
['Country code 3-character', "Student father's country of birth", 'REGION', "Student mother's country of birth", 'Country code 3-character', 'National Centre 6-digit Code', 'Adjudicated sub-region code 7-digit code (3-digit country code + region ID + stratum ID)', 'Unique national study programme code']
content="def run(df):\n    return float(df['CNT'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 289, 'total_tokens': 306}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6e6b7baf-9285-47ad-902e-8ce9699a4187-0' usage_metadata={'input_tokens': 289, 'output_tokens': 17, 'total_tokens': 306}
def run(df):
    return float(df['CNT'].dropna().nunique())


{'question': 'How many countries where in the survey?',
 'result': 80.0,
 'hint_cols': ['CNT',
  'COBN_F',
  'REGION',
  'COBN_M',
  'CNT',
  'NatCen',
  'SUBNATIO',
  'PROGN']}

In [112]:
utils.pipeline(llm, 'Which column stores student performance score?', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Student Name
3. Performance Score
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['AGE', 'COBN_S', 'ST004D01T', 'ST003D03T']
['AGE', 'COBN_S', 'ST004D01T', 'ST003D03T']
['PV3MPIN', 'PV3READ', 'PV4MPIN', 'PV3MPRE']
['PV3MPIN', 'PV3READ', 'PV4MPIN', 'PV3MPRE']
['Intl. Student ID', 'Intl. School ID', "Students' age", "Student's county of birth", "Students' age", "Student's county of birth", 'Student (Standardized) Gender', 'Student (Standardized) Birth -Year', 'Plausible Value 3 in Process Subscale of Mathematics - Interpreting, Applying, and Evaluating Mathematical Outcomes', 'Plausible Value 3 in Reading', 'Plausible Value 4 in Process Subscale of Mathematics - Interpreting, Applying, and Evaluating Mathematical Outcomes', 'Plausible Value 3 in Process Subscale of Mathematics - Reasoning']
content="def run(df):\n    return df[['PV3MPIN', 'PV3READ', 'PV4MPIN', 'PV3MPRE']].dropna().sum().sum()" response_metadata={'token_usage': {'compl

{'question': 'Which column stores student performance score?',
 'result': 1047417385.1069999,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'AGE',
  'COBN_S',
  'ST004D01T',
  'ST003D03T',
  'PV3MPIN',
  'PV3READ',
  'PV4MPIN',
  'PV3MPRE']}

In [117]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Social background (e.g. parental education level, household income)
3. Math score
hint 1. Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint 2. Social background (e.g. parental education level, household income)
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
hint 3. Math score
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
content="```python\nimport pandas as pd\nimport numpy as np\nfrom sklearn.linear_model import LinearRegression\n\ndef run(data):\n    data = data[['MISCED', 'FISCED', 'HISCED', 'PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']].dropna()\n    \n    X = data[['MISCED', 'FISCED', 'HISCED', 'MATHPREF']]\n    y = data['PV3MPRE']\n    \n    model = LinearRegression()\n    model.fit(X, y)\n    \n    return float(model.coef_[4])\n```" response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 461, 'total_tokens': 593}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-c937de8f-d4fb-4f

IndexError: index 4 is out of bounds for axis 0 with size 4

In [114]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.', df_sch, meta_sch, cols_retriever)

1. Student ID
2. Social background (e.g. parental education level, household income)
3. Math score

Linear regression coefficient: The coefficient will indicate the strength and direction of the relationship between social background and math score.
hint 1. Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint 2. Social background (e.g. parental education level, household income)
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
['MISCED', 'FISCED', 'HISCED', 'ATTIMMP']
hint 3. Math score
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
['PV3MPRE', 'MATHPREF', 'PV3MPFS', 'PV3MPIN']
hint 
['PV2SCIE', 'PV9SCIE', 'SISCO', 'PV2READ']
['PV2SCIE', 'PV9SCIE', 'SISCO', 'PV2READ']
hint Linear regression coefficient: The coefficient will indicate the strength and direction of the relationship between social background and math score.
['MATHMOT', 'PV1MCCR', 'PV4MCCR', 'PV2MCCR']
['MATHMOT', 'PV1MCCR', 'PV4MCCR', 'PV2MCCR']
['Intl. Student ID', 'Intl. School ID',

{'question': 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.',
 'result': 8.370679333883233,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'MISCED',
  'FISCED',
  'HISCED',
  'ATTIMMP',
  'PV3MPRE',
  'MATHPREF',
  'PV3MPFS',
  'PV3MPIN',
  'PV2SCIE',
  'PV9SCIE',
  'SISCO',
  'PV2READ',
  'MATHMOT',
  'PV1MCCR',
  'PV4MCCR',
  'PV2MCCR']}

In [118]:
utils.pipeline(llm, 'How many OECD member countries where in the survey? ', df_sch, meta_sch, cols_retriever)

- Country
- OECD Member (Yes/No)
hint - Country
['REGION', 'OECD', 'COBN_F', 'COBN_M']
hint - OECD Member (Yes/No)
['OECD', 'IMMIG', 'PA018Q02NA', 'ST330D10WA']
content="import pandas as pd\n\ndef run(data):\n    data.dropna(subset=['OECD'], inplace=True)\n    num_oecd_countries = len(data['OECD'].unique())\n    \n    return float(num_oecd_countries)" response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 319, 'total_tokens': 364}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5392d2f5-7d3d-4e21-bab1-36ed047cba10-0' usage_metadata={'input_tokens': 319, 'output_tokens': 45, 'total_tokens': 364}
import pandas as pd

def run(data):
    data.dropna(subset=['OECD'], inplace=True)
    num_oecd_countries = len(data['OECD'].unique())
    
    return float(num_oecd_countries)


{'question': 'How many OECD member countries where in the survey? ',
 'result': 2.0,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'COBN_M',
  'OECD',
  'IMMIG',
  'PA018Q02NA',
  'ST330D10WA']}

In [119]:
utils.pipeline(llm, 'How many countries that are OECD members where in the survey? ', df_sch, meta_sch, cols_retriever)

- Country Name
- OECD Member Status
hint - Country Name
['CNT', 'REGION', 'COBN_F', 'OECD']
hint - OECD Member Status
['OECD', 'ESCS', 'IMMIG', 'BMMJ1']
content="def run(df):\n    return float(df['OECD'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 285, 'total_tokens': 303}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-4fbad042-c710-4c51-a029-a97cae06eb02-0' usage_metadata={'input_tokens': 285, 'output_tokens': 18, 'total_tokens': 303}
def run(df):
    return float(df['OECD'].dropna().nunique())


{'question': 'How many countries that are OECD members where in the survey? ',
 'result': 2.0,
 'hint_cols': ['CNT',
  'REGION',
  'COBN_F',
  'OECD',
  'OECD',
  'ESCS',
  'IMMIG',
  'BMMJ1']}

In [122]:
utils.pipeline(llm, 'how many countries that are OECD memeber were involved in the survey?', df_sch, meta_sch, cols_retriever)

- Country
- OECD Member (Yes/No)
hint - Country
['REGION', 'OECD', 'COBN_F', 'COBN_M']
hint - OECD Member (Yes/No)
['OECD', 'IMMIG', 'PA018Q02NA', 'ST330D10WA']
content="def run(data):\n    return float(data['OECD'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 322, 'total_tokens': 340}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-bdb79af0-b6d7-457b-9775-930e1e727194-0' usage_metadata={'input_tokens': 322, 'output_tokens': 18, 'total_tokens': 340}
def run(data):
    return float(data['OECD'].dropna().nunique())


{'question': 'how many countries that are OECD memeber were involved in the survey?',
 'result': 2.0,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'COBN_M',
  'OECD',
  'IMMIG',
  'PA018Q02NA',
  'ST330D10WA']}

In [125]:
utils.pipeline(llm, 'how many countries (column name CNT) that are memeber of OECD organisation were involved in the survey?', df_sch, meta_sch, cols_retriever)

1. Country Name
2. OECD Member Status
3. Survey Participation Status
hint 1. Country Name
['CNT', 'COBN_F', 'REGION', 'COBN_M']
hint 2. OECD Member Status
['OECD', 'IMMIG', 'ESCS', 'BMMJ1']
hint 3. Survey Participation Status
['Option_WBQ', 'Option_PQ', 'Option_TQ', 'BSMJ']
content="def run(data):\n    return float(data['CNT'][data['OECD'] == 1].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 406, 'total_tokens': 432}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-634ec3fd-e830-4e28-9f1f-8c7860e1c0f1-0' usage_metadata={'input_tokens': 406, 'output_tokens': 26, 'total_tokens': 432}
def run(data):
    return float(data['CNT'][data['OECD'] == 1].dropna().nunique())


{'question': 'how many countries (column name CNT) that are memeber of OECD organisation were involved in the survey?',
 'result': 37.0,
 'hint_cols': ['CNT',
  'COBN_F',
  'REGION',
  'COBN_M',
  'OECD',
  'IMMIG',
  'ESCS',
  'BMMJ1',
  'Option_WBQ',
  'Option_PQ',
  'Option_TQ',
  'BSMJ']}

In [126]:
utils.pipeline(llm, 'how many different countries that are memeber of OECD organisation were involved in the survey?', df_sch, meta_sch, cols_retriever)

- Country
- OECD_member
- Survey_involved
hint - Country
['REGION', 'OECD', 'COBN_F', 'COBN_M']
hint - OECD_member
['OECD', 'IMMIG', 'OCOD3', 'OCOD1']
hint - Survey_involved
['LANGTEST_QQQ', 'Option_TQ', 'ADMINMODE', 'Option_WBQ']
content="def run(df):\n    return float(df['OECD'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 384, 'total_tokens': 402}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-98101135-4bf7-4ce5-867c-170a1dc83ac2-0' usage_metadata={'input_tokens': 384, 'output_tokens': 18, 'total_tokens': 402}
def run(df):
    return float(df['OECD'].dropna().nunique())


{'question': 'how many different countries that are memeber of OECD organisation were involved in the survey?',
 'result': 2.0,
 'hint_cols': ['REGION',
  'OECD',
  'COBN_F',
  'COBN_M',
  'OECD',
  'IMMIG',
  'OCOD3',
  'OCOD1',
  'LANGTEST_QQQ',
  'Option_TQ',
  'ADMINMODE',
  'Option_WBQ']}

In [148]:
utils.pipeline(llm, 'In how many languages ​​was the questionnaire filled out?', df_sch, meta_sch, cols_retriever)

1. Respondent ID
2. Language of questionnaire filled out
hint 1. Respondent ID
['ADMINMODE', 'BOOKID', 'CNTSTUID', 'CNTSCHID']
hint 2. Language of questionnaire filled out
['LANGTEST_QQQ', 'LANGTEST_PAQ', 'Option_TQ', 'Option_WBQ']
content="def run(data):\n    return float(data['LANGTEST_QQQ'].dropna().nunique())" response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 287, 'total_tokens': 307}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-9ec3f902-e705-4d61-acca-6344e15b67d0-0' usage_metadata={'input_tokens': 287, 'output_tokens': 20, 'total_tokens': 307}
def run(data):
    return float(data['LANGTEST_QQQ'].dropna().nunique())


{'question': 'In how many languages \u200b\u200bwas the questionnaire filled out?',
 'result': 55.0,
 'hint_cols': ['ADMINMODE',
  'BOOKID',
  'CNTSTUID',
  'CNTSCHID',
  'LANGTEST_QQQ',
  'LANGTEST_PAQ',
  'Option_TQ',
  'Option_WBQ']}

In [149]:
utils.pipeline(llm, 'In which country name have students the most books at home?', df_sch, meta_sch, cols_retriever)

- Student ID
- Country
- Number of books at home
hint - Student ID
['CNTSTUID', 'CNTSCHID', 'AGE', 'COBN_S']
hint - Country
['CNTRYID', 'REGION', 'OECD', 'COBN_F']
hint - Number of books at home
['PA195Q01JA', 'ST255Q01JA', 'ST256Q10JA', 'ST256Q06JA']
content="def run(data):\n    data.dropna(subset=['PA195Q01JA', 'ST255Q01JA'], inplace=True)\n    data['total_books'] = data['PA195Q01JA'] + data['ST255Q01JA']\n    country_with_most_books = data.groupby('COBN_S')['total_books'].sum().idxmax()\n    return float(country_with_most_books)" response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 410, 'total_tokens': 490}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-787e467b-b525-49c8-9fe9-c360e97ac93d-0' usage_metadata={'input_tokens': 410, 'output_tokens': 80, 'total_tokens': 490}
def run(data):
    data.dropna(subset=['PA195Q01JA', 'ST255Q01JA'], inplace=True)
    data['total_books'] = data['PA195Q01JA'] +

{'question': 'In which country name have students the most books at home?',
 'result': 38000.0,
 'hint_cols': ['CNTSTUID',
  'CNTSCHID',
  'AGE',
  'COBN_S',
  'CNTRYID',
  'REGION',
  'OECD',
  'COBN_F',
  'PA195Q01JA',
  'ST255Q01JA',
  'ST256Q10JA',
  'ST256Q06JA']}

In [3]:
utils.pipeline(llm, 'What is the relation between social background and math score of students? Please give linear regression coefficient as answer.', df_sch, meta_sch, cols_retriever)

NameError: name 'llm' is not defined